# Exploratory Data Analysis of traffic data at Road 51 Lauttasaari

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Data source: https://www.avoindata.fi/data/fi/dataset/liikennemaarat-helsingissa

Legend: ha=henkilöautot pa=pakettiautot ka=kuorma-autot ra=rekka-autot la=linja-autot mp=moottoripyörät rv=raitiovaunut

Laskenta on tehty tunneittain, paitsi ruuhka-aikoina (klo 6.00-9.00 ja 15.00-18.00) jaksotus on 15 minuuttia. Kellonajat ovat alkavia kellonaikoja. Laskentapisteiden poikkileikkaukset lasketaan suunnittain (suunta 1 on keskustaan, linjalla D D1-D13 länteen)

In [2]:
data = pd.read_csv('data/hki_liikennemaarat.csv', encoding='latin-1',delimiter=';')
data.head()

,piste,nimi,x_gk25,y_gk25,suunta,aika,vuosi,ha,pa,ka,ra,la,mp,rv,autot
0,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,0,2011,76,5,1,0,5,0,0,87
1,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,100,2011,65,5,1,0,4,0,0,75
2,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,200,2011,61,4,1,0,4,0,0,70
3,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,300,2011,52,4,1,0,3,0,0,60
4,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,400,2011,31,2,0,0,2,0,0,35


Remove data about trams and other measurement points.

In [3]:
lauttasaari = data[data.nimi == 'LAUTTASAAREN SILTA']
lauttasaari = lauttasaari.loc[:,['suunta','aika','vuosi','ha','pa','ka','ra','la','mp','autot']]

In [4]:
yearly = lauttasaari.groupby(['vuosi'])
hel_2011 = yearly.get_group(2011)
to_hel_2011 = hel_2011[hel_2011.suunta == 1]
cars_to_hel_2011 = to_hel_2011.loc[:,['aika','autot']]
cars_to_hel_2011

,aika,autot
0,0,87
1,100,75
2,200,70
3,300,60
4,400,35
5,500,87
6,600,54
7,615,60
8,630,77
9,645,96


Since the time series is at uneven intervals some reductions have to be made.

In [5]:
c_6to7 = cars_to_hel_2011[(cars_to_hel_2011['aika'] >= 600) & (cars_to_hel_2011['aika'] < 700)]
c_6to7

,aika,autot
6,600,54
7,615,60
8,630,77
9,645,96


In [6]:
c_6to7['autot'].sum()

287

In [7]:
cars_to_hel_2011.loc[6,['autot']] = c_6to7['autot'].sum()
cars_to_hel_2011.drop(cars_to_hel_2011.index[7:10],inplace=True)

In [8]:
cars_to_hel_2011

,aika,autot
0,0,87
1,100,75
2,200,70
3,300,60
4,400,35
5,500,87
6,600,287
10,700,61
11,715,81
12,730,148


In [10]:
c_7to8 = cars_to_hel_2011[(cars_to_hel_2011['aika'] >= 700) & (cars_to_hel_2011['aika'] < 800)]
c_7to8

,aika,autot
10,700,61
11,715,81
12,730,148
13,745,115


In [19]:
cars_to_hel_2011.at[10,['autot']] = c_7to8['autot'].sum()
cars_to_hel_2011.drop(cars_to_hel_2011.index[11:13],inplace=True)
cars_to_hel_2011

,aika,autot
0,0,87
1,100,75
2,200,70
3,300,60
4,400,35
5,500,87
6,600,287
10,700,405
11,715,81
12,730,148
